In [1]:
%load_ext autoreload
%autoreload 2

# Tor

If you run this crawler long enough, eventually you'll begin to encounter addresses that look like `'fd87:d87e:eb43:20e:46fd:450c:e42c:29a5'`

```
select * from nodes where ip like '%fd87:d87e:eb43%';
```

That curious 6 byte `fd87:d87e:eb43` prefix will show up far more than it should by chance alone.

Curiously, these prefixes never appear amonth the IPs where connections were successful, only among the unsuccessful connections.

Try connecting to a few and you'll just get timeouts

So what are they?

They're Tor nodes.

Here's how we can convert these 16 bytes into an "onion" address used by the Tor network:

In [2]:
import lib

ip = 'fd87:d87e:eb43:20e:46fd:450c:e42c:29a5'
ip_bytes = lib.ip_to_bytes(ip)

# First, strip the 6-byte prefix
onion = ip_bytes[6:]
onion

b'\x02\x0eF\xfdE\x0c\xe4,)\xa5'

In [3]:
from base64 import b32encode

# Base32 encode the bytes
onion = b32encode(onion)
onion

b'AIHEN7KFBTSCYKNF'

In [4]:
# Lowercase it (strictly speaking you don't need to do this)
onion = onion.lower()
onion

b'aihen7kfbtscyknf'

In [5]:
# Decode to a string and .onion to the end
onion = onion.decode() + ".onion"
onion

'aihen7kfbtscyknf.onion'

In [6]:
# all together now ...

def ip_bytes_to_onion(ip_bytes):
    return b32encode(ip_bytes[6:]).lower().decode("ascii") + ".onion"

ip_bytes_to_onion(ip_bytes)

'aihen7kfbtscyknf.onion'

In [7]:
# But we still can't connect to us from a trusty socket ...
import socket

socket.create_connection((onion, 8333))

KeyboardInterrupt: 

This is because we need to be running Tor locally and need to install a python package that can use tor as a proxy to make this connection

Go to the [Tor website](https://www.torproject.org/download/download-easy.html.en) and install Tor if you haven't already

Run this command in your terminal to check whether it's working:

```shell
$ curl --socks5 localhost:9050 --socks5-hostname localhost:9050 -s https://check.torproject.org/ | cat | grep -m 1 Congratulations | xargs
```

If everything is working, you should get a response declaring `Congratulations. This browser is configured to use Tor.`

Once you've done that, install the pysocks python proxy:

```shell
$ pip install PySocks 
```

Here's a demonstration showing, at the very least, the recipients of requests over this proxy no longer see your original IP address:

In [8]:
import socks
import urllib

print("Old IP", urllib.request.urlopen('http://icanhazip.com').read().decode().strip())
socks.setdefaultproxy(
    proxy_type=socks.PROXY_TYPE_SOCKS5, 
    addr="127.0.0.1", 
    port=9050,
)
socket.socket = socks.socksocket  # swap out socket.socket
print("New IP", urllib.request.urlopen('http://icanhazip.com').read().decode().strip())
import socket  # swap socket.socket back in ...

Old IP 2605:6000:1018:4d4:7993:66e3:34ff:18f1
New IP 158.69.217.87


And finally, let's acually connect to the tor bitcoin node:

In [9]:
sock = socks.create_connection(
    (onion, 8333),
    proxy_type=socks.PROXY_TYPE_SOCKS5,
    proxy_addr="127.0.0.1",
    proxy_port=9050
)
stream = sock.makefile('rb')

# Send "version"
payload = lib.serialize_version_payload()
msg = lib.serialize_msg(command=b"version", payload=payload)
sock.sendall(msg)

# Receive "version"
their_version = lib.read_msg(stream)
print(f"Received {their_version}")

# ... it works ...

Received {'command': b'version', 'payload': b"\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x00\x1aI\xac\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00&\xf50'\x85\x0b\x9e\xc2%/Satoshi:0.17.0(P2P Electronic Cash)/\xe8\xb5\x08\x00\x01"}
